In [1]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlrd
from openpyxl import load_workbook
import os
from datetime import datetime

In [2]:
rfid_coc = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_cocaine.csv', index_col=0)
rfid_oxy = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_oxycodone.csv', index_col=0)

In [3]:
characteristics_LGA_SHA = ['rfid','subject','room','cohort','trial_id','drug','box', 'start_time', 'end_time',
'start_date','end_date','active_lever_presses','inactive_lever_presses','reward_presses','timeout_presses',
'active_timestamps','inactive_timestamps','reward_timestamps','timeout_timestamps']

integer_col_LGA_SHA = ['cohort','active_lever_presses','inactive_lever_presses','reward_presses','timeout_presses']

In [4]:
# get all the sheetnames in one excel workbook
def get_sheetnames_xlsx(file_name):
    wb = load_workbook(file_name, read_only=True, keep_links=False)
    return wb.sheetnames

# clean subject id
def clean_subject_id(sid):
    sid = sid.upper()
    if 'F' in sid:
        char = 'F'
    if 'M' in sid:
        char = 'M'
        
    idx = sid.index(char)
    return sid[idx:].split('.')[0]

# convert column names into correct format
def clean_cols(s):
    if 'U' in s:
        return s.replace('U','Active ')
    elif 'V' in s:
        return s.replace('V','Inactive ')
    elif 'Y' in s:
        return s.replace('Y','Reward ')
    else:
        return s

In [111]:
OXY_PATH = '/Users/yunyihuang/Desktop/gl_data/GEORGE/OXY'
COC_PATH = '/Users/yunyihuang/Desktop/gl_data/GEORGE/COC'

In [180]:
input_path = OXY_PATH
workbooks = [os.path.join(input_path, i) for i in sorted(os.listdir(input_path)) if ('modified' in i)]
workbooks

['/Users/yunyihuang/Desktop/gl_data/GEORGE/OXY/C01GWASoxy_allnonMEDPCsessions_modified.xlsx']

In [181]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    # temporary
    worksheets = [ws for ws in worksheets if 'PR' not in ws]
    print(len(worksheets))
    for ws in worksheets:
        print(ws)
        transform_lga_sha(wb, ws, input_path)

28
C01HSOXYLGA01-20180814
C01HSOXYLGA02-20180815
C01HSOXYLGA03-20180817
C01HSOXYLGA04-20180818
C01HSOXYLGA05-20180821
C01HSOXYLGA07-20180823
C01HSOXYLGA08-20180824
C01HSOXYLGA09-20180825
C01HSOXYLGA10-20180828
C01HSOXYLGA11-20180829
C01HSOXYLGA12-20180830
C01HSOXYLGA13-20180831
C01HSOXYLGA14-20180901
C01HSOXYLGA15-20180905
C01HSOXYLGA16-20180907
C01HSOXYLGA17-20180911
C01HSOXYLGA18-20180913
C01HSOXYLGA19-20180915
C01HSOXYSHA01-20180730
C01HSOXYSHA02-20180731
C01HSOXYSHA03-20180801
C01HSOXYSHA04-20180802
C01HSOXYSHA05-20180803
C01HSOXYSHA06-20180806
C01HSOXYSHA07-20180807
C01HSOXYSHA08-20180808
C01HSOXYSHA09-20180809
C01HSOXYSHA10-20180810


In [157]:
wb = workbooks[1]
worksheets = sorted(get_sheetnames_xlsx(wb))
# temporary
worksheets = [ws for ws in worksheets if 'PR' not in ws]
ws = worksheets[0]

transform_lga_sha(wb, ws, input_path)

In [178]:
def transform_lga_sha(wb, ws, input_path):
    filepath = os.path.join(input_path, wb)
    df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()
    
    # modify the header
    new_header = df_raw.iloc[0]     #grab the first row for the header
    df = df_raw[1:]                 #take the data except the header row
    df.columns = new_header 
    
    # clean subject id
    ID_col = df.columns.tolist()[0]
    df[ID_col] = df[ID_col].apply(clean_subject_id)

    # get rid of 0s
    df.replace(0, np.nan, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.fillna(0,inplace=True)
    
    # transform columns names
    filtered_cols = [i for i in df.columns if i[0] in ['U','V','Y','T'] or 
                     i in [ID_col, 'Active Lever Presses', 'Inactive Lever Presses', 'Reward']]
    dff = df[filtered_cols]
    new_cols = [clean_cols(i) for i in dff.columns]
    dff.columns = new_cols
    
    # add extra info
    if 'OXY' in ws:
        drug = 'Oxycodone'
        parser = r'(C[0-9]{2})HSOXY((?:LGA|SHA)[0-9]{2})'
        rfid_to_merge = rfid_oxy
        if 'LGA' in ws:
            output_path = 'OXY_LGA'
        if 'SHA' in ws:
            output_path = 'OXY_SHA'
    else:
        drug = 'Cocaine'
        parser = r'(C[0-9]{2})HS((?:LGA|SHA)[0-9]{2})'
        rfid_to_merge = rfid_coc
        if 'LGA' in ws:
            output_path = 'COC_LGA'
        if 'SHA' in ws:
            output_path = 'COC_SHA'
        
    if '-' in ws:
        to_split = '-'
    if '_' in ws:
        to_split = '_'
    
    info, date = ws.split('.')[0].split(to_split)
    cohort, trial_id = re.findall(parser, ws)[0]
    dt = pd.to_datetime(date, format='%Y%m%d', errors='ignore')
    
    dff['room'] = ['N/A'] * len(dff)
    dff['cohort'] = [cohort[1:]] * len(dff)
    dff['trial_id'] = [trial_id] * len(dff)
    dff['drug'] = [drug] * len(dff)
    dff['box'] = [-999] * len(dff)
    dff['start_time'] = [datetime.min.time()] * len(dff)
    dff['end_time'] = [datetime.min.time()] * len(dff)
    dff['start_date'] = [dt] * len(dff)
    dff['end_date'] = [datetime.min.date()] * len(dff)
    
    # group the timestamps
    colnames = dff.columns.tolist()
    active_col_begin = colnames.index('Active 0')
    inactive_col_begin = colnames.index('Inactive 0')
    reward_col_begin = colnames.index('Reward 0')
    timeout_col_begin = colnames.index('Timeout Press 1')
    timeout_col_end = colnames.index('room')
    
    dff['Active Timestamps'] = dff.iloc[:, active_col_begin:inactive_col_begin].values.tolist()
    dff['Inactive Timestamps'] = dff.iloc[:, inactive_col_begin:reward_col_begin].values.tolist()
    dff['Reward Timestamps'] = dff.iloc[:, reward_col_begin:timeout_col_begin].values.tolist()
    dff['Timeout Timestamps'] = dff.iloc[:, timeout_col_begin:timeout_col_end].values.tolist()
    
    # reformat columns, merge rfid
    dff.drop(dff.iloc[:, active_col_begin:timeout_col_end], inplace=True, axis=1)
    dff.rename(columns={"Reward": "Reward Presses", ID_col:"subject"}, inplace=True)
    dff['Timeout Presses'] = dff['Timeout Timestamps'].apply(lambda x: len([i for i in x if i != 0]))
    dff = pd.merge(dff, rfid_to_merge,  how='left', on = ['subject'])
    dff.rename(columns=str.lower,inplace=True)
    dff.columns = dff.columns.str.replace(' ','_')
    dff.fillna({'rfid':-999}, inplace=True)
    dff = dff[characteristics_LGA_SHA]
    
    # casting data type
    for col in dff.columns.tolist():
        if col in integer_col_LGA_SHA:
            dff[col] = dff[col].astype('int32')
    
    # display(dff)
    filename = ws.split('.')[0] + '_transformed.csv'
    dff.to_csv(os.path.join(output_path, filename))

In [176]:
print(datetime.min.time())

00:00:00


In [136]:
wb = workbooks[0]
print(wb)
worksheets = sorted(get_sheetnames_xlsx(wb))
# temporary
worksheets = [ws for ws in worksheets if 'PR' not in ws]
worksheets

/Users/yunyihuang/Desktop/gl_data/GEORGE/COC/C01GWASCocaine_allnonMEDPCsessions_modified.xlsx


['C01HSLGA01-20170815.xlsx',
 'C01HSLGA02-20170816.xlsx',
 'C01HSLGA03-20170817.xlsx',
 'C01HSLGA04-20170818.xlsx',
 'C01HSLGA05-20170821.xlsx',
 'C01HSLGA06-20170822.xlsx',
 'C01HSLGA07-20170823.xlsx',
 'C01HSLGA08-20170824.xlsx',
 'C01HSLGA09-20170825.xlsx',
 'C01HSLGA10-20170828.xlsx',
 'C01HSLGA11-20170829.xlsx',
 'C01HSLGA12-20170830.xlsx',
 'C01HSLGA13-20170831.xlsx',
 'C01HSLGA14-20170901.xlsx',
 'C01HSLGA15-20170907.xlsx',
 'C01HSLGA16-20170912.xlsx',
 'C01HSLGA17-20170914.xlsx',
 'C01HSLGA18-20170918.xlsx',
 'C01HSSHA01-20170731.xlsx',
 'C01HSSHA02-20170801.xlsx',
 'C01HSSHA03-20170802.xlsx',
 'C01HSSHA04-20170803.xlsx',
 'C01HSSHA05-20170804.xlsx',
 'C01HSSHA06-20170807.xlsx',
 'C01HSSHA07-20170808.xlsx',
 'C01HSSHA08-20170809.xlsx',
 'C01HSSHA09_20170810.xlsx',
 'C01HSSHA10_20170811.xlsx',
 'C02HSSHA01-20171017.xlsx',
 'C02HSSHA02-20171018.xlsx',
 'C02HSSHA03-20171019.xlsx',
 'C02HSSHA04-20171020.xlsx',
 'C02HSSHA05-20171023.xlsx',
 'C02HSSHA06-20171024.xlsx',
 'C02HSSHA07-2

In [137]:
ws = worksheets[0]
ws

'C01HSLGA01-20170815.xlsx'

In [141]:
wb

'/Users/yunyihuang/Desktop/gl_data/GEORGE/COC/C01GWASCocaine_allnonMEDPCsessions_modified.xlsx'

In [146]:
transform_lga_sha(wb, ws, input_path)

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000120124704,F18,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,158,0,117,41,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97..."
1,933000120117306,F19,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,4,6,4,0,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,933000120117312,F20,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,104,0,98,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2..."
3,933000120117311,F21,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,180,4,129,51,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24..."
4,933000120117329,F22,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,191,0,115,76,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26..."
5,933000120117323,F23,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,4,1,2,2,"[358.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[2947.54, 17852.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[2947.54, 17852.09, 17858.31, 17871.05, 0.0, 0...","[17858.31, 17871.05, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,933000120117313,F26,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,172,0,107,65,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[1699.68, 1699.87, 1700.59, 1861.71, 1863.03, ..."
7,933000120124705,M1,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,33,19,22,11,"[58.88, 60.1, 86.56, 87.65, 97.24, 101.74, 362...","[4.78, 37.8, 112.22, 3478.98, 11613.16, 15589....","[4.78, 37.8, 40.09, 112.22, 3478.98, 11613.16,...","[40.09, 11617.11, 11621.74, 11622.57, 15599.87..."
8,933000120117352,M13,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,2,0,2,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,933000120117335,M14,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,0,6,0,0,"[53.53, 58.14, 10188.09, 12656.87, 12676.27, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [142]:
test = pd.read_excel(wb, sheet_name = ws).T.reset_index()

In [143]:
# modify the header
new_header = test.iloc[0]   #grab the first row for the header
df = test[1:]               #take the data except the header row
df.columns = new_header 
df.head()

,AID,Active Lever Presses,Inactive Lever Presses,Reward,L0,L1,L2,L3,L4,L5,...,Timeout Press 67,Timeout Press 68,Timeout Press 69,Timeout Press 70,Timeout Press 71,Timeout Press 72,Timeout Press 73,Timeout Press 74,Timeout Press 75,Timeout Press 76
1,F18,158.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F19,4.0,6.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F20,104.0,0.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,F21,180.0,4.0,129.0,4.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,F22,191.0,0.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16557.34,16559.52,16559.92,16733.94,17162.53,17814.91,17816.26,17922.12,20327.77,20612.33


In [144]:
# clean subject id
df['AID'] = df['AID'].apply(clean_subject_id)

# get rid of 0s
df.replace(0, np.nan, inplace=True)
df.dropna(how='all', axis=1, inplace=True)
df.fillna(0,inplace=True)
df.head()

,AID,Active Lever Presses,Inactive Lever Presses,Reward,L0,L1,L2,L3,L4,L5,...,Timeout Press 67,Timeout Press 68,Timeout Press 69,Timeout Press 70,Timeout Press 71,Timeout Press 72,Timeout Press 73,Timeout Press 74,Timeout Press 75,Timeout Press 76
1,F18,158.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,F19,4.0,6.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,F20,104.0,0.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,F21,180.0,4.0,129.0,4.0,2.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,F22,191.0,0.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16557.34,16559.52,16559.92,16733.94,17162.53,17814.91,17816.26,17922.12,20327.77,20612.33


In [119]:
# transform columns names
filtered_cols = [i for i in df.columns if i[0] in ['U','V','Y','T'] or i in ['AID', 'Active Lever Presses', 'Inactive Lever Presses', 'Reward']]
dff = df[filtered_cols]
new_cols = [clean_cols(i) for i in dff.columns]
dff.columns = new_cols
dff.head()

,AID,Active Lever Presses,Inactive Lever Presses,Reward,Active 0,Active 1,Active 2,Active 3,Active 4,Active 5,...,Timeout Press 67,Timeout Press 68,Timeout Press 69,Timeout Press 70,Timeout Press 71,Timeout Press 72,Timeout Press 73,Timeout Press 74,Timeout Press 75,Timeout Press 76
1,F18,158.0,0.0,117.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,F19,4.0,6.0,4.0,15529.67,15536.32,15620.29,15649.01,16287.25,20135.63,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,F20,104.0,0.0,98.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,F21,180.0,4.0,129.0,326.92,376.85,11614.59,13856.44,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,F22,191.0,0.0,115.0,0.00,0.00,0.00,0.00,0.00,0.00,...,16557.34,16559.52,16559.92,16733.94,17162.53,17814.91,17816.26,17922.12,20327.77,20612.33


In [120]:
if 'OXY' in ws:
    drug = 'Oxycodone'
    parser = r'(C[0-9]{2})HSOXY((?:LGA|SHA)[0-9]{2})'
    rfid_to_merge = rfid_oxy
else:
    drug = 'Cocaine'
    parser = r'(C[0-9]{2})HS((?:LGA|SHA)[0-9]{2})'
    rfid_to_merge = rfid_coc

In [121]:
# add extra session info
info, date = ws.split('.')[0].split('-')
cohort, trial_id = re.findall(parser, ws)[0]
dt = pd.to_datetime(date, format='%Y%m%d', errors='ignore')
print(cohort, trial_id, dt)

C01 LGA01 2017-08-15 00:00:00


In [122]:
dff['room'] = [None] * len(dff)
dff['cohort'] = [cohort[1:]] * len(dff)
dff['trial_id'] = [trial_id] * len(dff)
dff['drug'] = [drug] * len(dff)
dff['box'] = [None] * len(dff)
dff['start_time'] = [None] * len(dff)
dff['end_time'] = [None] * len(dff)
dff['start_date'] = [dt] * len(dff)
dff['end_date'] = [None] * len(dff)

dff.head()

,AID,Active Lever Presses,Inactive Lever Presses,Reward,Active 0,Active 1,Active 2,Active 3,Active 4,Active 5,...,Timeout Press 76,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date
1,F18,158.0,0.0,117.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None
2,F19,4.0,6.0,4.0,15529.67,15536.32,15620.29,15649.01,16287.25,20135.63,...,0.00,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None
3,F20,104.0,0.0,98.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None
4,F21,180.0,4.0,129.0,326.92,376.85,11614.59,13856.44,0.00,0.00,...,0.00,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None
5,F22,191.0,0.0,115.0,0.00,0.00,0.00,0.00,0.00,0.00,...,20612.33,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None


In [123]:
# group the timestamps
colnames = dff.columns.tolist()
active_col_begin = colnames.index('Active 0')
inactive_col_begin = colnames.index('Inactive 0')
reward_col_begin = colnames.index('Reward 0')
timeout_col_begin = colnames.index('Timeout Press 1')
timeout_col_end = colnames.index('room')

In [124]:
dff['Active Timestamps'] = dff.iloc[:, active_col_begin:inactive_col_begin].values.tolist()
dff['Inactive Timestamps'] = dff.iloc[:, inactive_col_begin:reward_col_begin].values.tolist()
dff['Reward Timestamps'] = dff.iloc[:, reward_col_begin:timeout_col_begin].values.tolist()
dff['Timeout Timestamps'] = dff.iloc[:, timeout_col_begin:timeout_col_end].values.tolist()

In [125]:
dff.head()

,AID,Active Lever Presses,Inactive Lever Presses,Reward,Active 0,Active 1,Active 2,Active 3,Active 4,Active 5,...,drug,box,start_time,end_time,start_date,end_date,Active Timestamps,Inactive Timestamps,Reward Timestamps,Timeout Timestamps
1,F18,158.0,0.0,117.0,0.00,0.00,0.00,0.00,0.00,0.00,...,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97..."
2,F19,4.0,6.0,4.0,15529.67,15536.32,15620.29,15649.01,16287.25,20135.63,...,Cocaine,None,None,None,2017-08-15,None,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,F20,104.0,0.0,98.0,0.00,0.00,0.00,0.00,0.00,0.00,...,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2..."
4,F21,180.0,4.0,129.0,326.92,376.85,11614.59,13856.44,0.00,0.00,...,Cocaine,None,None,None,2017-08-15,None,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24..."
5,F22,191.0,0.0,115.0,0.00,0.00,0.00,0.00,0.00,0.00,...,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26..."


In [127]:
dff.drop(dff.iloc[:, active_col_begin:timeout_col_end], inplace=True, axis=1)
dff.rename(columns={"Reward": "Reward Presses", "AID":"subject"}, inplace=True)
dff['Timeout Presses'] = dff['Timeout Timestamps'].apply(lambda x: len([i for i in x if i != 0]))
dff.head()

,subject,Active Lever Presses,Inactive Lever Presses,Reward Presses,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,Active Timestamps,Inactive Timestamps,Reward Timestamps,Timeout Timestamps,Timeout Presses
1,F18,158.0,0.0,117.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97...",41
2,F19,4.0,6.0,4.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,F20,104.0,0.0,98.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2...",6
4,F21,180.0,4.0,129.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24...",51
5,F22,191.0,0.0,115.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26...",76


In [128]:
dff = pd.merge(dff, rfid_to_merge,  how='left', on = ['subject'])
dff.head()

,subject,Active Lever Presses,Inactive Lever Presses,Reward Presses,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,Active Timestamps,Inactive Timestamps,Reward Timestamps,Timeout Timestamps,Timeout Presses,rfid
0,F18,158.0,0.0,117.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97...",41,933000120124704
1,F19,4.0,6.0,4.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,933000120117306
2,F20,104.0,0.0,98.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2...",6,933000120117312
3,F21,180.0,4.0,129.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24...",51,933000120117311
4,F22,191.0,0.0,115.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26...",76,933000120117329


In [129]:
dff.rename(columns=str.lower,inplace=True)
dff.columns = dff.columns.str.replace(' ','_')
dff.fillna({'rfid':-999}, inplace=True)
dff.head()

,subject,active_lever_presses,inactive_lever_presses,reward_presses,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps,timeout_presses,rfid
0,F18,158.0,0.0,117.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97...",41,933000120124704
1,F19,4.0,6.0,4.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,933000120117306
2,F20,104.0,0.0,98.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2...",6,933000120117312
3,F21,180.0,4.0,129.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24...",51,933000120117311
4,F22,191.0,0.0,115.0,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26...",76,933000120117329


In [130]:
dff = dff[characteristics_LGA_SHA]
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000120124704,F18,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,158.0,0.0,117.0,41,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97..."
1,933000120117306,F19,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,4.0,6.0,4.0,0,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,933000120117312,F20,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,104.0,0.0,98.0,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2..."
3,933000120117311,F21,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,180.0,4.0,129.0,51,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24..."
4,933000120117329,F22,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,191.0,0.0,115.0,76,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26..."
5,933000120117323,F23,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,4.0,1.0,2.0,2,"[358.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[2947.54, 17852.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[2947.54, 17852.09, 17858.31, 17871.05, 0.0, 0...","[17858.31, 17871.05, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,933000120117313,F26,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,172.0,0.0,107.0,65,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[1699.68, 1699.87, 1700.59, 1861.71, 1863.03, ..."
7,933000120124705,M1,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,33.0,19.0,22.0,11,"[58.88, 60.1, 86.56, 87.65, 97.24, 101.74, 362...","[4.78, 37.8, 112.22, 3478.98, 11613.16, 15589....","[4.78, 37.8, 40.09, 112.22, 3478.98, 11613.16,...","[40.09, 11617.11, 11621.74, 11622.57, 15599.87..."
8,933000120117352,M13,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,2.0,0.0,2.0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,933000120117335,M14,None,01,LGA01,Cocaine,None,None,None,2017-08-15,None,0.0,6.0,0.0,0,"[53.53, 58.14, 10188.09, 12656.87, 12676.27, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [131]:
integer_col_LGA_SHA = ['cohort','active_lever_presses','inactive_lever_presses','reward_presses','timeout_presses']

In [132]:
for col in dff.columns.tolist():
    if col in integer_col_LGA_SHA:
        dff[col] = dff[col].astype('int32')

In [133]:
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000120124704,F18,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,158,0,117,41,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[122.12, 173.98, 200.39, 574.74, 648.2, 674.35...","[122.12, 173.98, 200.39, 201.22, 574.74, 648.2...","[201.22, 2076.87, 4745.52, 8216.1, 9558.92, 97..."
1,933000120117306,F19,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,4,6,4,0,"[15529.67, 15536.32, 15620.29, 15649.01, 16287...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[15346.87, 15741.93, 17279.53, 17440.92, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,933000120117312,F20,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,104,0,98,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[460.33, 486.89, 507.17, 592.48, 665.39, 750.3...","[824.27, 3527.1, 4077.55, 7435.76, 14555.08, 2..."
3,933000120117311,F21,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,180,4,129,51,"[326.92, 376.85, 11614.59, 13856.44, 0.0, 0.0,...","[300.43, 352.74, 381.85, 540.62, 594.35, 697.6...","[300.43, 303.28, 352.74, 381.85, 540.62, 594.3...","[303.28, 594.82, 1166.42, 1300.74, 1459.68, 24..."
4,933000120117329,F22,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,191,0,115,76,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[63.81, 98.67, 131.34, 158.2, 430.69, 472.37, ...","[63.81, 83.44, 98.67, 112.94, 131.34, 158.2, 4...","[83.44, 112.94, 472.57, 475.62, 477.55, 484.26..."
5,933000120117323,F23,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,4,1,2,2,"[358.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[2947.54, 17852.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[2947.54, 17852.09, 17858.31, 17871.05, 0.0, 0...","[17858.31, 17871.05, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,933000120117313,F26,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,172,0,107,65,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[685.13, 730.81, 1060.53, 1260.34, 1406.96, 16...","[1699.68, 1699.87, 1700.59, 1861.71, 1863.03, ..."
7,933000120124705,M1,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,33,19,22,11,"[58.88, 60.1, 86.56, 87.65, 97.24, 101.74, 362...","[4.78, 37.8, 112.22, 3478.98, 11613.16, 15589....","[4.78, 37.8, 40.09, 112.22, 3478.98, 11613.16,...","[40.09, 11617.11, 11621.74, 11622.57, 15599.87..."
8,933000120117352,M13,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,2,0,2,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[4444.48, 4490.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,933000120117335,M14,None,1,LGA01,Cocaine,None,None,None,2017-08-15,None,0,6,0,0,"[53.53, 58.14, 10188.09, 12656.87, 12676.27, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [148]:
len(dff.columns)

19

In [147]:
dff['timeout_timestamps'].values[3]

[303.28,
 594.82,
 1166.42,
 1300.74,
 1459.68,
 2454.75,
 3109.78,
 3113.45,
 3113.63,
 4051.17,
 4059.52,
 5044.93,
 5584.49,
 6758.44,
 7500.13,
 8064.87,
 8399.31,
 8609.45,
 8830.9,
 8831.55,
 8843.01999999999,
 8843.49999999999,
 8844.2,
 8845.59999999999,
 8845.92999999999,
 8846.51999999999,
 9037.23,
 9206.71999999999,
 9207.31999999999,
 9207.58999999999,
 9208.37,
 9214.2,
 9214.74999999999,
 9270.00999999999,
 10310.96,
 11902.62,
 12719.6,
 12795.38,
 12796.14,
 13320.84,
 13853.55,
 14598.7,
 16536.68,
 16538.01,
 16538.5,
 16538.86,
 16726.39,
 16726.91,
 17744.79,
 19907.47,
 21241.87,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [307]:
filename = ws + '_transformed.csv'
dff.to_csv(filename)

In [165]:
ws + '_transformed.csv'

'C05HSSHA10-20180810_transformed.csv'

In [ ]:
'C05HSSHA10-20180810.xlsx'